In [1]:
import pandas as pd
import numpy as np
import cv2 
import glob
from natsort import natsorted
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.models import load_model
from sklearn.metrics import mean_squared_error, r2_score
from natsort import natsorted
import os, sys
%matplotlib inline
# demonstrate data standardization with sklearn
from sklearn.preprocessing import StandardScaler
# physical_devices = tf.config.list_physical_devices('GPU') 
# for device in physical_devices:
#     tf.config.experimental.set_memory_growth(device, True)


import torch
# import torch.nn as nn
from torch import nn
# from tab_transformer_pytorch import TabTransformer  
torch.backends.cudnn.enabled = True

In [2]:
# Check if GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("PyTorch is running on GPU")
else:
    device = torch.device("cpu")
    print("PyTorch is running on CPU")

PyTorch is running on GPU


In [3]:
case_no ="045"

In [4]:
# Read the Excel file
df = pd.read_excel('Areas_of _echo.xlsx')

In [5]:
# Display the first five rows of the data
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(df)


    RV_Study_ID  FourC_ED  FourC_ES  PLAX_ED  PLAX_ES  PSAX_AV_ED  PSAX_AV_ES  \
0             1     13.54      5.70     6.94     3.13       23.95       14.35   
1             2     16.05      5.45     8.46     4.45       16.33        8.68   
2             3     10.76      4.52     6.67     3.54       13.87        6.07   
3             4     18.39     12.33    12.32     5.77       26.27       15.35   
4             5     20.36     11.83    11.87     7.16       32.87       18.25   
5             6     10.68      5.97     8.64     4.90       17.08        7.19   
6             7     22.62     14.50    11.77     8.03       21.46       13.70   
7             8     13.08      8.76    17.18     9.57       16.61        8.59   
8             9     20.81     11.80     8.71     4.64       17.35       10.12   
9            10     13.57      4.47    14.00     6.20       15.98        2.72   
10           11     16.80      7.59    14.01     6.10       26.62       14.48   
11           12      7.75   

In [6]:
FourC = pd.concat([df["FourC_ED"], df["FourC_ES"]])
PLAX =pd.concat([df["PLAX_ED"], df["PLAX_ES"]])
PSAX_AV =pd.concat([df["PSAX_AV_ED"], df["PSAX_AV_ES"]])
PSAXbase =pd.concat([df["PSAXbase_ED"], df["PSAXbase_ES"]])
PSAXdistal =pd.concat([df["PSAXdistal_ED"], df["PSAXdistal_ES"]])
PSAXmid =pd.concat([df["PSAXmid_ED"], df["PSAXmid_ES"]])
Rinflow =pd.concat([df["Rinflow_ED"], df["Rinflow_ES"]])
SubC =pd.concat([df["SubC_ED"], df["SubC_ES"]])


img_FourC = pd.concat([df["img_FourC_ED"], df["img_FourC_ES"]])
img_PLAXD =pd.concat([df["img_PLAX_ED"], df["img_PLAX_ES"]])
img_PSAX_AV =pd.concat([df["img_PSAX_AV_ED"], df["img_PSAX_AV_ES"]])
img_PSAXbase =pd.concat([df["img_PSAXbase_ED"], df["img_PSAXbase_ES"]])
img_PSAXdistal =pd.concat([df["img_PSAXdistal_ED"], df["img_PSAXdistal_ES"]])
img_PSAXmid =pd.concat([df["img_PSAXmid_ED"], df["img_PSAXmid_ES"]])
img_Rinflow =pd.concat([df["img_Rinflow_ED"], df["img_Rinflow_ES"]])
img_SubC =pd.concat([df["img_SubC_ED"], df["img_SubC_ED"]])

In [7]:
df1 = pd.read_excel('RV project patient list.xlsx')

In [8]:
df1.head()

,RV_Study_ID,Gender,Age_at_earliest_study,CMR_RVEDV,CMR_RVESV,CMR_RVEF,ED,ES
0,1,0,56,119,52,56,0,1
1,2,0,53,114,40,65,0,1
2,3,0,42,110,43,61,0,1
3,4,1,61,177,67,62,0,1
4,5,1,27,212,110,48,0,1


In [9]:
RVEDV = df1["CMR_RVEDV"]
RVESV = df1["CMR_RVESV"]
RVEF  = df1["CMR_RVEF"]
Phase = pd.concat([df1["ED"], df1["ES"]])
Gender= pd.concat([df1["Gender"], df1["Gender"]])
Age = pd.concat([df1["Age_at_earliest_study"], df1["Age_at_earliest_study"]])

In [10]:
img_FourC_train, img_FourC_test = img_FourC[10:-10].tolist(), img_FourC[:10].tolist() + img_FourC[50:60].tolist() 

FourC_train, FourC_test =  FourC[10:-10].tolist(), FourC[:10].tolist() + FourC[50:60].tolist() 
PLAX_train, PLAX_test = PLAX[10:-10].tolist(), PLAX[:10].tolist() + PLAX[50:60].tolist() 
PSAX_AV_train, PSAX_AV_test = PSAX_AV[10:-10].tolist(), PSAX_AV[:10].tolist() + PSAX_AV[50:60].tolist() 
PSAXbase_train, PSAXbase_test = PSAXbase[10:-10].tolist(), PSAXbase[:10].tolist() + PSAXbase[50:60].tolist() 
PSAXdistal_train, PSAXdistal_test = PSAXdistal[10:-10].tolist(), PSAXdistal[:10].tolist() + PSAXdistal[50:60].tolist() 
PSAXmid_train, PSAXmid_test= PSAXmid[10:-10].tolist(), PSAXmid[:10].tolist() + PSAXmid[50:60].tolist() 
Rinflow_train, Rinflow_test = Rinflow[10:-10].tolist(), Rinflow[:10].tolist() + Rinflow[50:60].tolist() 
SubC_train, SubC_test = SubC[10:-10].tolist(), SubC[:10].tolist() + SubC[50:60].tolist() 
Gender_train, Gender_test  = Gender[10:-10].tolist(), Gender[:10].tolist() + Gender[50:60].tolist() 
Age_train, Age_test  = Age[10:-10].tolist(), Age[:10].tolist() + Age[50:60].tolist() 
Phase_train, Phase_test  = Phase[10:-10].tolist(), Phase[:10].tolist() + Phase[50:60].tolist() 


In [11]:
X_train = []
for i in range(len(FourC_train)):
    X_train.append(np.array([Phase_train[i], Gender_train[i], Age_train[i], FourC_train[i], PLAX_train[i], PSAX_AV_train[i], PSAXbase_train[i], PSAXdistal_train[i], PSAXmid_train[i], Rinflow_train[i], SubC_train[i]]))
    
X_train = np.array(X_train, dtype='float32')    

In [12]:
X_test = []
for i in range(len(FourC_test)):
    X_test.append(np.array([Phase_test[i], Gender_test[i], Age_test[i], FourC_test[i], PLAX_test[i], PSAX_AV_test[i], PSAXbase_test[i], PSAXdistal_test[i], PSAXmid_test[i], Rinflow_test[i], SubC_test[i]]))
    
X_test = np.array(X_test, dtype='float32')  

In [13]:
RV_vol = pd.concat([df1["CMR_RVEDV"], df1["CMR_RVESV"]])

RV_vol_train, RV_vol_test =  RV_vol[10:-10].tolist(), RV_vol[:10].tolist() + RV_vol[50:60].tolist() 

y_train = np.array(RV_vol_train, dtype='float32')

y_test = np.array(RV_vol_test, dtype='float32')

In [14]:
y_test

array([119., 114., 110., 177., 212.,  97., 126., 188., 188., 124.,  52.,
        40.,  43.,  67., 110.,  46.,  66., 100.,  88.,  46.],
      dtype=float32)

In [15]:
# created scaler
scaler = StandardScaler()
# fit scaler on training dataset
scaler.fit(y_train.reshape(-1, 1))
# transform training dataset
y_train = scaler.transform(y_train.reshape(-1, 1))
# # transform test dataset
# y_test = scaler.transform(y_test.reshape(-1, 1))

In [16]:
X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])


print(X_train.shape)

(80, 1, 11)


In [17]:
X_train[0]

array([[ 0.  ,  0.  , 65.  , 16.8 , 14.01, 26.62,  9.44,  1.8 ,  5.77,
        25.06, 10.21]], dtype=float32)

In [18]:

from tab_transformer_pytorch import FTTransformer

model = FTTransformer(
    categories = (2, 2),      # tuple containing the number of unique values within each category
    num_continuous = 9,                # number of continuous values
    dim = 16,                           # dimension, paper set at 32
    dim_out = 1,                        # binary prediction, but could be anything
    depth = 3,                          # depth, paper recommended 6
    heads = 24,                          # heads, paper recommends 8
    attn_dropout = 0.3,                 # post-attention dropout
    ff_dropout = 0.3                    # feed forward dropout
)

# cont_mean_std = torch.randn(11, 2)
# x_categ = torch.randint(0, 5, (1, 5))     # category values, from 0 - max number of categories, in the order as passed into the constructor above
# x_cont = torch.randn(1, 11)               # assume continuous values are already normalized individually

# pred = model(x_categ, x_cont).to(device) # (1, 1)

x_categ = torch.randint(0, 5, (1, 5)) 

x_categ

Ix_categ = torch.Tensor([[2, 2, 5, 5, 5, 5, 5, 5, 5, 5, 5]]).int()

Ix_categ

from torchsummary import summary


model = model.to(device)

summary(model,((1, 5),(1, 10)))

In [19]:
print(model)

FTTransformer(
  (categorical_embeds): Embedding(6, 16)
  (numerical_embedder): NumericalEmbedder()
  (transformer): Transformer(
    (layers): ModuleList(
      (0): ModuleList(
        (0): Attention(
          (norm): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
          (to_qkv): Linear(in_features=16, out_features=1152, bias=False)
          (to_out): Linear(in_features=384, out_features=16, bias=False)
          (dropout): Dropout(p=0.3, inplace=False)
        )
        (1): Sequential(
          (0): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
          (1): Linear(in_features=16, out_features=128, bias=True)
          (2): GEGLU()
          (3): Dropout(p=0.3, inplace=False)
          (4): Linear(in_features=64, out_features=16, bias=True)
        )
      )
      (1): ModuleList(
        (0): Attention(
          (norm): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
          (to_qkv): Linear(in_features=16, out_features=1152, bias=False)
          (to_

In [20]:
from tensorflow.keras import backend as K


def r2_torch_score(y_pred, y_true):
    mean_y_true = torch.mean(y_true)
    SS_res = torch.sum((y_true - y_pred) ** 2)
    SS_tot = torch.sum((y_true - mean_y_true) ** 2)
    r2 = 1 - (SS_res / SS_tot)
    return r2
#     return tf.clip_by_value(r2, 0, 1)

def r2_torch_loss(y_pred, y_true):
    mean_y_true = torch.mean(y_true)
    SS_res = torch.sum((y_true - y_pred) ** 2)
    SS_tot = torch.sum((y_true - mean_y_true) ** 2)
    r2 = 1 - (SS_res / SS_tot)
    return 1-r2
#     return tf.clip_by_value((1-r2), 0, 1)


def mape_loss(y_true, y_pred):
    
    if len(y_pred.shape)>=3:
        y_pred = tf.squeeze(y_pred, axis=-1)
        y_pred = tf.squeeze(y_pred, axis=-1)
    if len(y_pred.shape)>=2:
        y_pred = tf.squeeze(y_pred, axis=-1)
        
        
    abs_diff = tf.abs(y_true-y_pred)/y_true
    return tf.reduce_mean(abs_diff)*100



In [21]:
from torch.utils.data import TensorDataset, DataLoader




# Define the loss function and optimizer
# criterion = nn.MSELoss()
criterion = nn.L1Loss()

optimizer = torch.optim.Adamax(model.parameters(), lr=0.01)

input_data = torch.Tensor(X_train)
labels = torch.Tensor(y_train)

In [22]:
import random


# Divide dataset into training and validation splits
num_samples = len(input_data)
indices = list(range(num_samples))
random.shuffle(indices)
split = int(num_samples * 0.975)  # 80% for training and 20% for validation  0.975
train_idx, val_idx = indices[:split], indices[split:]
train_inputs = input_data[train_idx]
train_labels = labels[train_idx]
val_inputs = input_data[val_idx]
val_labels = labels[val_idx]

In [23]:
# Create TensorDatasets for training and validation
train_dataset = TensorDataset(train_inputs, train_labels)
val_dataset = TensorDataset(val_inputs, val_labels)

# Define DataLoaders
batch_size = 1  # choose a batch size
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


In [24]:
import time
import warnings

warnings.filterwarnings("ignore")
model = model.to(device)

# Loop over the epochs
num_epochs = 500

begin_time = time.time()
for epoch in range(num_epochs):
    start_time = time.time()
    # Training
    model.train()
    for inputs, labels in train_dataloader:
        tr_pred =[]
        tr_true =[]
        optimizer.zero_grad()
        inputs = inputs.squeeze(0).squeeze(0).to(device)
        labels = labels.to(device)
        x_categ = torch.Tensor([[int(inputs[0]),int(inputs[1])]]).long().to(device)
        x_numec = torch.Tensor([[float(inputs[2]),float(inputs[3]),float(inputs[4]),float(inputs[5]),float(inputs[6]),float(inputs[7]),float(inputs[8]),float(inputs[9]),float(inputs[10])]]).to(device)
        outputs = model(x_categ,x_numec)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        tr_pred.append(outputs[0][0].data.detach().cpu().numpy())
        tr_true.append(labels[0].data.detach().cpu().numpy())
    
            
    # Validation
    model.eval()
    with torch.no_grad():
        val_loss = 0.0
        val_loss_r2 = 0.0
        tst_pred =[]
        tst_true =[]
        for inputs, labels in val_dataloader:
            inputs = inputs.squeeze(0).squeeze(0).to(device)
            labels = labels.to(device)
            x_categ_val = torch.Tensor([[int(inputs[0]),int(inputs[1])]]).long().to(device)
            x_numec_val = torch.Tensor([[float(inputs[2]),float(inputs[3]),float(inputs[4]),float(inputs[5]),float(inputs[6]),float(inputs[7]),float(inputs[8]),float(inputs[9]),float(inputs[10])]]).to(device)
            outputs = model(x_categ_val, x_numec_val)
            val_loss += criterion(outputs, labels).item() * inputs.shape[0]
#             val_loss_r2 += r2_torch_score(outputs[0][0], labels[0]).item() * inputs.shape[0]
            tr_pred.append(outputs[0][0].data.detach().cpu().numpy())
            tr_true.append(labels[0].data.detach().cpu().numpy())
        val_loss /= len(val_dataset)
        val_loss_r2 /= len(val_dataset)
        
    loss_r2 =( 1- r2_score(tr_true, tr_pred))
    score_r2 = r2_score(tr_true, tr_pred)
    
    end_time = time.time()
    epoch_time = end_time - start_time
    
    # Save the model after each epoch
    model_file = './weight/'+case_no+'/'+ f"model_epoch_{epoch + 1}_{score_r2:.4f}.pth"
    torch.save(model, model_file)
    
    
    # Print results
    print(f"Epoch [{epoch + 1}/{num_epochs}], Train Loss: {loss.item():.4f}, Train R2 Loss: {loss_r2:.4f}, Val Loss: {val_loss:.4f}, R2 Val : {score_r2:.4f}",f" Time for epoch {epoch + 1}: {epoch_time:.2f} seconds")
finish_time = time.time()    
tot_time = finish_time - begin_time 
print(f"Total time {tot_time}")

Epoch [1/500], Train Loss: 0.0177, Train R2 Loss: 0.4913, Val Loss: 3.1018, R2 Val : 0.5087  Time for epoch 1: 2.26 seconds
Epoch [2/500], Train Loss: 0.1999, Train R2 Loss: 0.2303, Val Loss: 2.7277, R2 Val : 0.7697  Time for epoch 2: 1.69 seconds
Epoch [3/500], Train Loss: 0.4216, Train R2 Loss: 0.4160, Val Loss: 3.6238, R2 Val : 0.5840  Time for epoch 3: 1.69 seconds
Epoch [4/500], Train Loss: 0.2689, Train R2 Loss: 0.4090, Val Loss: 2.4630, R2 Val : 0.5910  Time for epoch 4: 1.68 seconds
Epoch [5/500], Train Loss: 0.1728, Train R2 Loss: 0.2369, Val Loss: 2.2107, R2 Val : 0.7631  Time for epoch 5: 1.69 seconds
Epoch [6/500], Train Loss: 0.1458, Train R2 Loss: 0.1737, Val Loss: 2.3944, R2 Val : 0.8263  Time for epoch 6: 1.67 seconds
Epoch [7/500], Train Loss: 0.0090, Train R2 Loss: 0.2523, Val Loss: 2.0683, R2 Val : 0.7477  Time for epoch 7: 1.67 seconds
Epoch [8/500], Train Loss: 2.7285, Train R2 Loss: 0.6455, Val Loss: 1.6624, R2 Val : 0.3545  Time for epoch 8: 1.67 seconds
Epoch [9

Epoch [67/500], Train Loss: 0.1955, Train R2 Loss: 0.1785, Val Loss: 1.8656, R2 Val : 0.8215  Time for epoch 67: 1.68 seconds
Epoch [68/500], Train Loss: 0.3299, Train R2 Loss: 0.1687, Val Loss: 2.2242, R2 Val : 0.8313  Time for epoch 68: 1.67 seconds
Epoch [69/500], Train Loss: 0.0258, Train R2 Loss: 0.4188, Val Loss: 1.9613, R2 Val : 0.5812  Time for epoch 69: 1.69 seconds
Epoch [70/500], Train Loss: 0.1033, Train R2 Loss: 0.0872, Val Loss: 1.8056, R2 Val : 0.9128  Time for epoch 70: 1.68 seconds
Epoch [71/500], Train Loss: 0.0106, Train R2 Loss: 0.3652, Val Loss: 1.8596, R2 Val : 0.6348  Time for epoch 71: 1.67 seconds
Epoch [72/500], Train Loss: 0.1703, Train R2 Loss: 0.2605, Val Loss: 2.8478, R2 Val : 0.7395  Time for epoch 72: 1.69 seconds
Epoch [73/500], Train Loss: 0.1548, Train R2 Loss: 0.2097, Val Loss: 3.7761, R2 Val : 0.7903  Time for epoch 73: 1.69 seconds
Epoch [74/500], Train Loss: 0.3764, Train R2 Loss: 0.2157, Val Loss: 2.5882, R2 Val : 0.7843  Time for epoch 74: 1.67 

Epoch [132/500], Train Loss: 0.2059, Train R2 Loss: 0.6432, Val Loss: 2.3578, R2 Val : 0.3568  Time for epoch 132: 1.67 seconds
Epoch [133/500], Train Loss: 0.1650, Train R2 Loss: 0.5509, Val Loss: 2.1256, R2 Val : 0.4491  Time for epoch 133: 1.68 seconds
Epoch [134/500], Train Loss: 0.1714, Train R2 Loss: 0.2650, Val Loss: 2.3392, R2 Val : 0.7350  Time for epoch 134: 1.67 seconds
Epoch [135/500], Train Loss: 0.0107, Train R2 Loss: 0.4554, Val Loss: 2.0749, R2 Val : 0.5446  Time for epoch 135: 1.67 seconds
Epoch [136/500], Train Loss: 0.0534, Train R2 Loss: 0.1291, Val Loss: 2.2602, R2 Val : 0.8709  Time for epoch 136: 1.68 seconds
Epoch [137/500], Train Loss: 0.1006, Train R2 Loss: 0.4942, Val Loss: 1.9952, R2 Val : 0.5058  Time for epoch 137: 1.67 seconds
Epoch [138/500], Train Loss: 0.2347, Train R2 Loss: 0.5111, Val Loss: 2.3487, R2 Val : 0.4889  Time for epoch 138: 1.67 seconds
Epoch [139/500], Train Loss: 0.3309, Train R2 Loss: 0.4685, Val Loss: 2.2817, R2 Val : 0.5315  Time for 

Epoch [196/500], Train Loss: 0.0097, Train R2 Loss: 0.0970, Val Loss: 1.3241, R2 Val : 0.9030  Time for epoch 196: 1.74 seconds
Epoch [197/500], Train Loss: 0.0425, Train R2 Loss: 0.2377, Val Loss: 1.7642, R2 Val : 0.7623  Time for epoch 197: 1.69 seconds
Epoch [198/500], Train Loss: 0.0255, Train R2 Loss: 0.0827, Val Loss: 1.4434, R2 Val : 0.9173  Time for epoch 198: 1.68 seconds
Epoch [199/500], Train Loss: 0.0434, Train R2 Loss: 0.1157, Val Loss: 1.5073, R2 Val : 0.8843  Time for epoch 199: 1.70 seconds
Epoch [200/500], Train Loss: 0.3298, Train R2 Loss: 0.5589, Val Loss: 3.2219, R2 Val : 0.4411  Time for epoch 200: 1.68 seconds
Epoch [201/500], Train Loss: 0.0832, Train R2 Loss: 0.0666, Val Loss: 1.4353, R2 Val : 0.9334  Time for epoch 201: 1.69 seconds
Epoch [202/500], Train Loss: 0.2806, Train R2 Loss: 0.3387, Val Loss: 1.2694, R2 Val : 0.6613  Time for epoch 202: 1.68 seconds
Epoch [203/500], Train Loss: 0.1140, Train R2 Loss: 0.4208, Val Loss: 1.8747, R2 Val : 0.5792  Time for 

Epoch [261/500], Train Loss: 0.3390, Train R2 Loss: 0.1957, Val Loss: 1.4478, R2 Val : 0.8043  Time for epoch 261: 1.84 seconds
Epoch [262/500], Train Loss: 0.0031, Train R2 Loss: 0.1317, Val Loss: 1.4286, R2 Val : 0.8683  Time for epoch 262: 1.75 seconds
Epoch [263/500], Train Loss: 0.1958, Train R2 Loss: 0.3100, Val Loss: 1.9069, R2 Val : 0.6900  Time for epoch 263: 1.70 seconds
Epoch [264/500], Train Loss: 0.0178, Train R2 Loss: 0.0388, Val Loss: 1.2905, R2 Val : 0.9612  Time for epoch 264: 1.69 seconds
Epoch [265/500], Train Loss: 0.0535, Train R2 Loss: 0.0909, Val Loss: 1.3689, R2 Val : 0.9091  Time for epoch 265: 1.74 seconds
Epoch [266/500], Train Loss: 0.0872, Train R2 Loss: 0.0486, Val Loss: 1.3982, R2 Val : 0.9514  Time for epoch 266: 1.73 seconds
Epoch [267/500], Train Loss: 0.0345, Train R2 Loss: 0.2144, Val Loss: 2.5819, R2 Val : 0.7856  Time for epoch 267: 1.70 seconds
Epoch [268/500], Train Loss: 0.0401, Train R2 Loss: 0.0801, Val Loss: 1.2874, R2 Val : 0.9199  Time for 

Epoch [326/500], Train Loss: 0.1387, Train R2 Loss: 0.1617, Val Loss: 1.3453, R2 Val : 0.8383  Time for epoch 326: 1.68 seconds
Epoch [327/500], Train Loss: 0.1367, Train R2 Loss: 0.0860, Val Loss: 1.7048, R2 Val : 0.9140  Time for epoch 327: 1.68 seconds
Epoch [328/500], Train Loss: 0.3262, Train R2 Loss: 0.1346, Val Loss: 1.2458, R2 Val : 0.8654  Time for epoch 328: 1.67 seconds
Epoch [329/500], Train Loss: 0.3096, Train R2 Loss: 0.1396, Val Loss: 1.3066, R2 Val : 0.8604  Time for epoch 329: 1.68 seconds
Epoch [330/500], Train Loss: 0.0603, Train R2 Loss: 0.1172, Val Loss: 1.5295, R2 Val : 0.8828  Time for epoch 330: 1.68 seconds
Epoch [331/500], Train Loss: 0.1066, Train R2 Loss: 0.1188, Val Loss: 1.2446, R2 Val : 0.8812  Time for epoch 331: 1.68 seconds
Epoch [332/500], Train Loss: 0.1458, Train R2 Loss: 0.1420, Val Loss: 1.6291, R2 Val : 0.8580  Time for epoch 332: 1.68 seconds
Epoch [333/500], Train Loss: 0.0268, Train R2 Loss: 0.1170, Val Loss: 1.3498, R2 Val : 0.8830  Time for 

Epoch [391/500], Train Loss: 0.1563, Train R2 Loss: 0.0350, Val Loss: 1.0344, R2 Val : 0.9650  Time for epoch 391: 1.67 seconds
Epoch [392/500], Train Loss: 0.0567, Train R2 Loss: 0.0234, Val Loss: 0.8866, R2 Val : 0.9766  Time for epoch 392: 1.67 seconds
Epoch [393/500], Train Loss: 0.0442, Train R2 Loss: 0.0598, Val Loss: 1.0409, R2 Val : 0.9402  Time for epoch 393: 1.68 seconds
Epoch [394/500], Train Loss: 0.1280, Train R2 Loss: 0.0448, Val Loss: 1.0259, R2 Val : 0.9552  Time for epoch 394: 1.69 seconds
Epoch [395/500], Train Loss: 0.0371, Train R2 Loss: 0.0697, Val Loss: 0.9640, R2 Val : 0.9303  Time for epoch 395: 1.68 seconds
Epoch [396/500], Train Loss: 0.1105, Train R2 Loss: 0.0703, Val Loss: 1.8096, R2 Val : 0.9297  Time for epoch 396: 1.73 seconds
Epoch [397/500], Train Loss: 0.0433, Train R2 Loss: 0.0521, Val Loss: 1.1225, R2 Val : 0.9479  Time for epoch 397: 1.73 seconds
Epoch [398/500], Train Loss: 0.0513, Train R2 Loss: 0.0474, Val Loss: 1.0746, R2 Val : 0.9526  Time for 

Epoch [456/500], Train Loss: 0.1247, Train R2 Loss: 0.0342, Val Loss: 0.9970, R2 Val : 0.9658  Time for epoch 456: 1.70 seconds
Epoch [457/500], Train Loss: 0.2055, Train R2 Loss: 0.0687, Val Loss: 0.9159, R2 Val : 0.9313  Time for epoch 457: 1.68 seconds
Epoch [458/500], Train Loss: 0.1349, Train R2 Loss: 0.0239, Val Loss: 0.7158, R2 Val : 0.9761  Time for epoch 458: 1.69 seconds
Epoch [459/500], Train Loss: 0.0929, Train R2 Loss: 0.0313, Val Loss: 0.9039, R2 Val : 0.9687  Time for epoch 459: 1.69 seconds
Epoch [460/500], Train Loss: 0.0074, Train R2 Loss: 0.0256, Val Loss: 0.7286, R2 Val : 0.9744  Time for epoch 460: 1.69 seconds
Epoch [461/500], Train Loss: 0.0897, Train R2 Loss: 0.0427, Val Loss: 0.9175, R2 Val : 0.9573  Time for epoch 461: 1.68 seconds
Epoch [462/500], Train Loss: 0.0573, Train R2 Loss: 0.0282, Val Loss: 0.9204, R2 Val : 0.9718  Time for epoch 462: 1.68 seconds
Epoch [463/500], Train Loss: 0.0440, Train R2 Loss: 0.0353, Val Loss: 0.8774, R2 Val : 0.9647  Time for 

In [25]:
X_test[0].shape

(1, 11)

In [26]:
# Make predictions
model.eval()
prediction = []
for i in range(X_test.shape[0]):
    x_categ_val = torch.Tensor([[X_test[i][0][0],X_test[i][0][1]]]).long().to(device)
    x_num_val = torch.Tensor([[X_test[i][0][2],X_test[i][0][3],X_test[i][0][4],X_test[i][0][5],X_test[i][0][6],X_test[i][0][7],X_test[i][0][8],X_test[i][0][9],X_test[i][0][10]]]).to(device)
#     inputs = torch.Tensor(X_test[i]).to(device)
    y_pred = model(x_categ_val,x_num_val)
    y_pred = scaler.inverse_transform(y_pred[0][0].data.detach().cpu().numpy().reshape(-1, 1))
    print(y_pred[0][0])

    prediction.append(y_pred[0][0])


90.69155
140.02486
95.362915
169.54025
192.97713
98.15507
154.1566
184.76297
173.84525
156.75713
48.594982
35.9335
39.826572
61.444508
108.296
43.359364
63.850708
103.053
82.808784
39.783978


In [27]:
y_pred = np.array(prediction)

In [28]:
if len(y_pred.shape)>=3:
    y_pred = tf.squeeze(y_pred, axis=-1)
    y_pred = tf.squeeze(y_pred, axis=-1)

# Compare the predictions with the ground truth
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print('\033[91m Mean Squared Error: ', mse)
print('\033[91m R-squared: ', r2)

 Mean Squared Error:  217.34428
 R-squared:  0.9175391711787944


In [29]:

for i,j in zip(y_test, y_pred):
    try:
        print(i,":==:",j)
    except:
        print(i,":==:",j.numpy())   
    
errors = np.abs(np.array(y_test) - np.array(y_pred))
mean_absolute_error = np.mean(errors)
mape = mape_loss(y_test, y_pred).numpy()

print("mean absolute error:", mean_absolute_error)
print("absolute_mean_percentage_error:", mape)

119.0 :==: 90.69155
114.0 :==: 140.02486
110.0 :==: 95.362915
177.0 :==: 169.54025
212.0 :==: 192.97713
97.0 :==: 98.15507
126.0 :==: 154.1566
188.0 :==: 184.76297
188.0 :==: 173.84525
124.0 :==: 156.75713
52.0 :==: 48.594982
40.0 :==: 35.9335
43.0 :==: 39.826572
67.0 :==: 61.444508
110.0 :==: 108.296
46.0 :==: 43.359364
66.0 :==: 63.850708
100.0 :==: 103.053
88.0 :==: 82.808784
46.0 :==: 39.783978
mean absolute error: 10.60341
absolute_mean_percentage_error: 9.885715


In [30]:
weights = natsorted(glob.glob('./weight/'+case_no+'/*'))

In [38]:
# weights[240]
weights[216]

'./weight/045\\model_epoch_217_0.6175.pth'

In [39]:
model_best = model = torch.load(weights[216]).to(device)

In [40]:
# Make predictions
model_best.eval()
prediction = []
for i in range(X_test.shape[0]):
    x_categ_val = torch.Tensor([[X_test[i][0][0],X_test[i][0][1]]]).long().to(device)
    x_num_val = torch.Tensor([[X_test[i][0][2],X_test[i][0][3],X_test[i][0][4],X_test[i][0][5],X_test[i][0][6],X_test[i][0][7],X_test[i][0][8],X_test[i][0][9],X_test[i][0][10]]]).to(device)
#     inputs = torch.Tensor(X_test[i]).to(device)
    y_pred = model(x_categ_val,x_num_val)
#     y_pred = model_best(x_categ_val,inputs)
    y_pred = scaler.inverse_transform(y_pred[0][0].data.detach().cpu().numpy().reshape(-1, 1))
    print(y_pred[0][0])

    prediction.append(y_pred[0][0])


98.986534
119.88882
99.927826
175.2175
204.8465
100.03241
138.45464
191.19417
184.19427
148.2673
54.739704
43.36855
50.66457
65.73337
113.441414
51.24905
59.24619
104.34665
92.07425
46.013683


In [41]:
y_pred = np.array(prediction)

In [42]:
if len(y_pred.shape)>=3:
    y_pred = tf.squeeze(y_pred, axis=-1)
    y_pred = tf.squeeze(y_pred, axis=-1)
# Compare the predictions with the ground truth
mse = mean_squared_error(y_test, y_pred)
# r2 = np.corrcoef(y_pred, y_test)[0, 1] ** 2
r2 = r2_score(y_test, y_pred)

print('\033[91m Mean Squared Error: ', mse)
print('\033[91m R-squared: ', r2)

 Mean Squared Error:  78.43117
 R-squared:  0.9702430680383609


In [43]:
for i,j in zip(y_test, y_pred):
    try:
        print(i,":==:",j)
    except:
        print(i,":==:",j.numpy())
errors = np.abs(np.array(y_test) - np.array(y_pred))
mean_absolute_error = np.mean(errors)
mape = mape_loss(y_test, y_pred).numpy()
# mape = tf.keras.losses.MAPE(y_test, y_pred).numpy()


print("mean absolute error:", mean_absolute_error)
# print("absolute_mean_percentage_error:", absolute_mean_percentage_error)
print("absolute_percentage_errors:", mape)

119.0 :==: 98.986534
114.0 :==: 119.88882
110.0 :==: 99.927826
177.0 :==: 175.2175
212.0 :==: 204.8465
97.0 :==: 100.03241
126.0 :==: 138.45464
188.0 :==: 191.19417
188.0 :==: 184.19427
124.0 :==: 148.2673
52.0 :==: 54.739704
40.0 :==: 43.36855
43.0 :==: 50.66457
67.0 :==: 65.73337
110.0 :==: 113.441414
46.0 :==: 51.24905
66.0 :==: 59.24619
100.0 :==: 104.34665
88.0 :==: 92.07425
46.0 :==: 46.013683
mean absolute error: 6.529151
absolute_percentage_errors: 6.950502


In [37]:
for j in range(len(weights)):
    model_new = model = torch.load(weights[j]).to(device)
# Make predictions
    model_new.eval()
    prediction = []
    for i in range(X_test.shape[0]):
        x_categ_val = torch.Tensor([[X_test[i][0][0],X_test[i][0][1]]]).long().to(device)
        x_num_val = torch.Tensor([[X_test[i][0][2],X_test[i][0][3],X_test[i][0][4],X_test[i][0][5],X_test[i][0][6],X_test[i][0][7],X_test[i][0][8],X_test[i][0][9],X_test[i][0][10]]]).to(device)
#     inputs = torch.Tensor(X_test[i]).to(device)
        y_pred = model(x_categ_val,x_num_val)
#     y_pred = model_best(x_categ_val,inputs)
        y_pred = scaler.inverse_transform(y_pred[0][0].data.detach().cpu().numpy().reshape(-1, 1))
        prediction.append(y_pred[0][0])
    if len(y_pred.shape)>=3:
        y_pred = tf.squeeze(y_pred, axis=-1)
        y_pred = tf.squeeze(y_pred, axis=-1)
    y_pred = np.array(prediction)
    score = r2_score(y_test,y_pred)
    print("model",int(j+1)," : ",round(score,4))

model 1  :  0.5258
model 2  :  0.7313
model 3  :  0.5839
model 4  :  0.6474
model 5  :  0.7641
model 6  :  0.8255
model 7  :  0.7496
model 8  :  0.8667
model 9  :  0.7336
model 10  :  0.7732
model 11  :  0.7393
model 12  :  0.7625
model 13  :  0.5374
model 14  :  0.7698
model 15  :  0.8012
model 16  :  0.8422
model 17  :  0.8043
model 18  :  0.7722
model 19  :  0.8885
model 20  :  0.8472
model 21  :  0.7971
model 22  :  0.5677
model 23  :  0.9067
model 24  :  0.8688
model 25  :  0.8312
model 26  :  0.8909
model 27  :  0.6968
model 28  :  0.8332
model 29  :  0.8598
model 30  :  0.9057
model 31  :  0.8545
model 32  :  0.91
model 33  :  0.9634
model 34  :  0.9115
model 35  :  0.9389
model 36  :  0.7164
model 37  :  0.8631
model 38  :  0.8219
model 39  :  0.8917
model 40  :  0.9108
model 41  :  0.9109
model 42  :  0.9303
model 43  :  0.8894
model 44  :  0.9406
model 45  :  0.8218
model 46  :  0.9303
model 47  :  0.8146
model 48  :  0.9324
model 49  :  0.9215
model 50  :  0.923
model 51  : 

model 402  :  0.93
model 403  :  0.924
model 404  :  0.9407
model 405  :  0.9311
model 406  :  0.8068
model 407  :  0.8635
model 408  :  0.9391
model 409  :  0.9241
model 410  :  0.8969
model 411  :  0.9092
model 412  :  0.9378
model 413  :  0.8926
model 414  :  0.953
model 415  :  0.9249
model 416  :  0.9374
model 417  :  0.9229
model 418  :  0.9125
model 419  :  0.9282
model 420  :  0.9267
model 421  :  0.9324
model 422  :  0.9
model 423  :  0.8964
model 424  :  0.929
model 425  :  0.9205
model 426  :  0.9106
model 427  :  0.8629
model 428  :  0.9155
model 429  :  0.8575
model 430  :  0.8887
model 431  :  0.9131
model 432  :  0.8823
model 433  :  0.8622
model 434  :  0.9164
model 435  :  0.9176
model 436  :  0.9234
model 437  :  0.9102
model 438  :  0.9096
model 439  :  0.8966
model 440  :  0.9172
model 441  :  0.9136
model 442  :  0.907
model 443  :  0.9056
model 444  :  0.8604
model 445  :  0.9286
model 446  :  0.8917
model 447  :  0.9088
model 448  :  0.9016
model 449  :  0.9269
m